In [15]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

#A base de dados está em: https://www.kaggle.com/spscientist/students-performance-in-exams
caminho_dataset = "StudentsPerformance.csv"
dados = pd.read_csv(caminho_dataset) 

#transformando "math_score" em classes de avaliação
def math_score_to_class(x):
    if x <= 33:
        return 0
    elif 34 <= x <= 66:
        return 1
    else:
        return 2

dados["math score"] = dados["math score"].apply(math_score_to_class)


X = dados.to_numpy()
y = X[:,5:6]
X = X[:,:5]

y = y.astype('int')
y = y.ravel()

label_encoder = LabelEncoder()

#transformando "Gender" em número inteiro
X[:,0:1] = label_encoder.fit_transform(X[:,0:1]).reshape(X.shape[0], 1)

#transformando "race/ethnicity" em número inteiro
X[:,1:2] = label_encoder.fit_transform(X[:,1:2]).reshape(X.shape[0], 1)

#transformando "parental level of education" em número inteiro
X[:,2:3] = label_encoder.fit_transform(X[:,2:3]).reshape(X.shape[0], 1)

#transformando "lunch" em número inteiro
X[:,3:4] = label_encoder.fit_transform(X[:,3:4]).reshape(X.shape[0], 1)

#transformando "test preparation course" em número inteiro
X[:,4:5] = label_encoder.fit_transform(X[:,4:5]).reshape(X.shape[0], 1)
X.shape, y.shape

((1000, 5), (1000,))

# Usando apenas Validação Cruzada - KNN e Logistic Regression

In [16]:
classificador_knn = KNeighborsRegressor()

scores_cv = cross_validate(classificador_knn, X, y, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False), cv=10)
print(scores_cv['test_score'])
sempad = np.mean(scores_cv['test_score'])
print(f"Sem padronização: {sempad}")

[-0.60860496 -0.53216539 -0.50990195 -0.54       -0.48124838 -0.51884487
 -0.57306195 -0.53478968 -0.59025418 -0.53851648]
Sem padronização: -0.5427387841645965


In [17]:
classificador_logreg = LogisticRegression()

scores_cv = cross_validate(classificador_logreg, X, y, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False), cv=10)
print(scores_cv['test_score'])
sempad = np.mean(scores_cv['test_score'])
print(f"Sem padronização: {sempad}")

[-0.6        -0.58309519 -0.6244998  -0.58309519 -0.58309519 -0.53851648
 -0.60827625 -0.6164414  -0.65574385 -0.6       ]
Sem padronização: -0.5992763354763799


# Usando Pipeline(com padronização dos dados) dentro do Grid Search - KNN e Logistic Regression

In [18]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", classificador_knn)
])

parametros = {'knn__n_neighbors': [3,5,7]}

grid_search_pipeline = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

scores_cv_gs_pipe = cross_validate(grid_search_pipeline, X, y, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False), cv=10)
print(scores_cv_gs_pipe['test_score'])
compad = np.mean(scores_cv_gs_pipe['test_score'])
print(f"Erro quadrático médio: {compad}")

[-0.59556182 -0.49010619 -0.51269596 -0.53832696 -0.46445052 -0.53623784
 -0.5331845  -0.52779866 -0.54697685 -0.53107515]
Erro quadrático médio: -0.5276414454843148


In [19]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("logreg", classificador_logreg)
])


parametros = {'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

grid_search_pipeline = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

scores_cv_gs_pipe = cross_validate(grid_search_pipeline, X, y, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False), cv=10)
print(scores_cv_gs_pipe['test_score'])
compad = np.mean(scores_cv_gs_pipe['test_score'])
print(f"Erro quadrático médio: {compad}")

[-0.6        -0.58309519 -0.6244998  -0.58309519 -0.58309519 -0.53851648
 -0.60827625 -0.6164414  -0.65574385 -0.6       ]
Erro quadrático médio: -0.5992763354763799


O melhor desempenho foi do KNN, com a padronização dos dados sendo feita dentro do Pipeline, que por sua vez está dentro do Grid Search.